In [1]:
import numpy as np
import pandas as pd
import re
from pyvi import ViTokenizer
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, SimpleRNN, Dropout, Convolution1D, Bidirectional

## Load data

In [2]:
train = pd.read_excel('Data/data_aivivn.xlsx')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16087 entries, 0 to 16086
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Label     16087 non-null  int64 
 1   Sentence  16087 non-null  object
dtypes: int64(1), object(1)
memory usage: 251.5+ KB


In [4]:
count_sentiment = pd.value_counts(train['Label'])
count_sentiment

0    8882
1    7205
Name: Label, dtype: int64

## Preprocessing data

In [5]:
def pre_processing(text):
    
    # Chuyển thành chữ thường
    text = text.lower()
    
    # Chuan hoa tieng viet, tieng anh, thuat ngu
    replace_list={
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
    'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
    'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
    'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
    'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
    #Quy các icon về 2 loại emoj: Tích cực hoặc tiêu cực
    "👹": "nagative", "👻": "positive", "💃": "positive",'🤙': ' positive ', '👍': ' positive ',
    "💄": "positive", "💎": "positive", "💩": "positive","😕": "nagative", "😱": "nagative", "😸": "positive",
    "😾": "nagative", "🚫": "nagative",  "🤬": "nagative","🧚": "positive", "🧡": "positive",'🐶':' positive ',
    '👎': ' nagative ', '😣': ' nagative ','✨': ' positive ', '❣': ' positive ','☀': ' positive ',
    '♥': ' positive ', '🤩': ' positive ', 'like': ' positive ', '💌': ' positive ',
    '🤣': ' positive ', '🖤': ' positive ', '🤤': ' positive ', ':(': ' nagative ', '😢': ' nagative ',
    '❤': ' positive ', '😍': ' positive ', '😘': ' positive ', '😪': ' nagative ', '😊': ' positive ',
    '?': ' ? ', '😁': ' positive ', '💖': ' positive ', '😟': ' nagative ', '😭': ' nagative ',
    '💯': ' positive ', '💗': ' positive ', '♡': ' positive ', '💜': ' positive ', '🤗': ' positive ',
    '^^': ' positive ', '😨': ' nagative ', '☺': ' positive ', '💋': ' positive ', '👌': ' positive ',
    '😖': ' nagative ', '😀': ' positive ', ':((': ' nagative ', '😡': ' nagative ', '😠': ' nagative ',
    '😒': ' nagative ', '🙂': ' positive ', '😏': ' nagative ', '😝': ' positive ', '😄': ' positive ',
    '😙': ' positive ', '😤': ' nagative ', '😎': ' positive ', '😆': ' positive ', '💚': ' positive ',
    '✌': ' positive ', '💕': ' positive ', '😞': ' nagative ', '😓': ' nagative ', '️🆗️': ' positive ',
    '😉': ' positive ', '😂': ' positive ', ':v': '  positive ', '=))': '  positive ', '😋': ' positive ',
    '💓': ' positive ', '😐': ' nagative ', ':3': ' positive ', '😫': ' nagative ', '😥': ' nagative ',
    '😃': ' positive ', '😬': ' 😬 ', '😌': ' 😌 ', '💛': ' positive ', '🤝': ' positive ', '🎈': ' positive ',
    '😗': ' positive ', '🤔': ' nagative ', '😑': ' nagative ', '🔥': ' nagative ', '🙏': ' nagative ',
    '🆗': ' positive ', '😻': ' positive ', '💙': ' positive ', '💟': ' positive ',
    '😚': ' positive ', '❌': ' nagative ', '👏': ' positive ', ';)': ' positive ', '<3': ' positive ',
    '🌝': ' positive ',  '🌷': ' positive ', '🌸': ' positive ', '🌺': ' positive ',
    '🌼': ' positive ', '🍓': ' positive ', '🐅': ' positive ', '🐾': ' positive ', '👉': ' positive ',
    '💐': ' positive ', '💞': ' positive ', '💥': ' positive ', '💪': ' positive ',
    '💰': ' positive ',  '😇': ' positive ', '😛': ' positive ', '😜': ' positive ',
    '🙃': ' positive ', '🤑': ' positive ', '🤪': ' positive ','☹': ' nagative ',  '💀': ' nagative ',
    '😔': ' nagative ', '😧': ' nagative ', '😩': ' nagative ', '😰': ' nagative ', '😳': ' nagative ',
    '😵': ' nagative ', '😶': ' nagative ', '🙁': ' nagative ',
    #Chuẩn hóa 1 số sentiment words/English words
    ':))': '  positive ', ':)': ' positive ', 'ô kêi': ' ok ', 'okie': ' ok ', ' o kê ': ' ok ',
    'okey': ' ok ', 'ôkê': ' ok ', ' oki ': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
    ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ', 'cam on':u'cám ơn',u'cảm ơn':'cám ơn',
    ' not ': u' không ',' khoong ': ' không ', u' kg ': u' không ','ko ': 'không', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ','hok':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' k ':u' không ',u'chẳng':u'không',u'đéo':u'không',
    'he he': ' positive ','hehe': ' positive ','hihi': ' positive ', 'haha': ' positive ', 'hjhj': ' positive ',
    ' lol ': ' negative ',' cc ': ' negative ','cute': u' dễ thương ','huhu': ' negative ', ' vs ': u' với ', 'wa': ' quá ', 'wá': u' quá', ' j ': u' gì ', '“': ' ',
    ' sz ': u' cỡ ', 'size': u' cỡ ', u' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
    'đc': u' được ','authentic': u' chuẩn chính hãng ',u' aut ': u' chuẩn chính hãng ', u' auth ': u' chuẩn chính hãng ', 'thick': u' positive ', 'store': u' cửa hàng ',
    'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
    'sấu': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
    'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
    'ể': 'ể', 'product': 'sản phẩm', 'quality': 'chất lượng','chat':' chất ', 'excelent': 'hoàn hảo', 'bad': 'tệ','fresh': ' tươi ','sad': ' tệ ',
    'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
    'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
    'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
    'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ', ' quả ng ': ' quảng ',
    'dep': u' đẹp ',u'xầu':u'xấu',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u' qủa': u' quả', 
    ' iu ': u' yêu ','fake': u' giả mạo ', 'trl': 'trả lời', '><': u' positive ', 
    ' por ': u' tệ ',' poor ': u' tệ ', 'ib':u' nhắn tin ', 'rep':u' trả lời ',u'fback':' feedback ','fedback':' feedback ',
    'qc':u'quảng cáo',u' éo ':' negative ',' bik ':' biết ',
    #dưới 3* quy về 1*, trên 3* quy về 5*
    '6 sao': ' 5star ','6 star': ' 5star ', '5star': ' 5star ','5 sao': ' 5star ','5sao': ' 5star ', '5 sao': '5star',
    'starstarstarstarstar': ' 5star ', '1 sao': ' 1star ', '1sao': ' 1star ','2 sao':' 1star ','2sao':' 1star ', '3 sao':'5star','4 sao':'5star','1 sao':'1star',
    '2 starstar':' 1star ','1star': ' 1star ', '0 sao': ' 1star ', '0star': ' 1star ','1*':' 1star ', '2*':' 1star ','3*':' 5star ','4*':' 5star ','5*':' 5star '
    }
    for k, v in replace_list.items():
        text = text.replace(k, v)
        
    # Xóa icon
    text = re.sub(r"\W", " ",text)
    
    # Xóa dấu câu
    text = re.sub('['+string.punctuation+']', ' ', text)
    
    # Tokenizer
    text = ViTokenizer.tokenize(text)
    return text

In [6]:
New_Sentence = []
for i in range(train.shape[0]):
    New_Sentence.append(pre_processing(train['Sentence'][i]))
train['New_Sentence'] = New_Sentence

In [7]:
train.head()

,Label,Sentence,New_Sentence
0,0,"""dung dc sp tot cam on shop đóng gói sản phẩm ...",dung được sản_phẩm tốt cám_ơn cửa_hàng đóng_gó...
1,0,""" chất lượng sản phẩm tuyệt vời . son mịn nhưn...",chất_lượng sản_phẩm tuyệt_vời son mịn nhưng kh...
2,0,""" chất lượng sản phẩm tuyệt vời nhưng k có hộp...",chất_lượng sản_phẩm tuyệt_vời nhưng không có h...
3,1,""":(( mình hơi thất vọng 1 chút vì mình đã kỳ v...",nagative mình hơi thất_vọng 1 chút vì mình đã ...
4,1,"""lần trước mình mua áo gió màu hồng rất ok mà ...",lần trước mình mua áo_gió màu hồng rất ok mà đ...


## Split data

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train_val, X_test, y_train_val, y_test = train_test_split(train.loc[:,'New_Sentence'].values, train.loc[:,'Label'].values, test_size=0.2, random_state=0)

In [10]:
x_train, x_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=0)

In [11]:
print(x_train.shape)
print(x_val.shape)
print(X_test.shape)

(10295,)
(2574,)
(3218,)


## Token and Vectorizer

In [12]:
vocab_size = 10000
embedding_dim = 64
max_length = 100
trunc_type = 'post'
padding_type = 'post'
oov_tok = '<OOV>'

In [13]:
# Tạo Tokenizer
tokenizer = Tokenizer(num_words= vocab_size, oov_token= oov_tok)

# Đưa từ vào tokenizer để tạo từ điển
tokenizer.fit_on_texts(x_train)

# Biến các câu train thành sequences
x_train = tokenizer.texts_to_sequences(x_train)

# Padding cho các train sequences
x_train = pad_sequences(x_train, maxlen= max_length, padding=padding_type, truncating= trunc_type)

# Biến các câu val thành sequences tương ứng
x_val = tokenizer.texts_to_sequences(x_val)

x_val = pad_sequences(x_val, maxlen= max_length, padding=padding_type, truncating= trunc_type)

# Biến các câu test thành sequences tương ứng
X_test = tokenizer.texts_to_sequences(X_test)

X_test = pad_sequences(X_test, maxlen= max_length, padding=padding_type, truncating= trunc_type)

## Buil Model

In [14]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=x_train.shape[1]))
model.add(Bidirectional(LSTM(32)))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 64)           640000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 16)                1040      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 665,889
Trainable params: 665,889
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [16]:
model.fit(x_train, y_train, epochs=4, validation_data=(x_val, y_val))

Epoch 1/4
322/322 [==============================] - 16s 49ms/step - loss: 0.3000 - accuracy: 0.8645 - val_loss: 0.1716 - val_accuracy: 0.9332
Epoch 2/4
322/322 [==============================] - 16s 49ms/step - loss: 0.1335 - accuracy: 0.9490 - val_loss: 0.1731 - val_accuracy: 0.9308
Epoch 3/4
322/322 [==============================] - 15s 47ms/step - loss: 0.0866 - accuracy: 0.9694 - val_loss: 0.1891 - val_accuracy: 0.9293
Epoch 4/4
322/322 [==============================] - 15s 46ms/step - loss: 0.0594 - accuracy: 0.9814 - val_loss: 0.2221 - val_accuracy: 0.9289


In [17]:
loss, acc = model.evaluate(X_test, y_test)

101/101 [==============================] - 1s 9ms/step - loss: 0.2273 - accuracy: 0.9313


In [66]:
model.save('model.h5')

In [70]:
model = load_model('model.h5')

In [18]:
t = ['thế thì chịu rồi']
t = tokenizer.texts_to_sequences(t)
t = pad_sequences(t, maxlen= max_length, padding=padding_type, truncating= trunc_type)
result = model.predict(t)
if result <=0.5: 
    print('tích cực')
else:
    print('tiêu cực')

tiêu cực
